In [1]:
import numpy as np
import pandas as pd
import re
import pickle
import string

In [2]:
with open('../static/model/model.pickle', 'rb') as f:
    model = pickle.load(f)

In [3]:
vocab = pd.read_csv('../static/model/vocablary.txt', header=None)
tokens = vocab[0].tolist()

In [4]:
txt = 'great product, i like it '

In [5]:
def remove_punctuations(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')
    return text

In [6]:
with open ('../static/model/corpora/stopwords/english', 'r') as file:
    sw = file.read().splitlines()

In [7]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()

In [8]:
def preprocessing(text):
    data = pd.DataFrame([text], columns=['tweet'])
    data['tweet'] = data['tweet'].apply(lambda x: " ".join(x.lower() for x in x.split()))
    data['tweet'] = data['tweet'].apply(lambda x: " ".join(re.sub(r'^https?:\/\/.*[\r\n]*', '' , x, flags=re.MULTILINE)  for x in x.split()))
    data["tweet"] = data["tweet"].apply(remove_punctuations)
    data["tweet"] = data["tweet"].str.replace('\d+', '', regex=True)
    data["tweet"] = data["tweet"].apply(lambda x: " ".join(x for x in x.split() if x not in sw))
    data["tweet"] = data["tweet"].apply(lambda x: " ".join(ps.stem(x) for x in x.split()))

    return data["tweet"]

In [9]:
def vectorizer(ds, vocablary):
    vectorized_lst = []

    for sentence in ds:
        sentence_lst = np.zeros(len(vocablary)) #replace all features with zeros

        for i in range (len(vocablary)): #if the faeature and the vocablary is matching it will replace with 1
            if vocablary[i] in sentence.split():
                sentence_lst[i] = 1
        vectorized_lst.append(sentence_lst) #append the list
    
    vectorized_lst_new = np.asarray(vectorized_lst, dtype=np.float32) #convert it to numpy array

    return vectorized_lst_new

In [10]:
def get_prediction(vectorized_text):
    prediction = model.predict(vectorized_text)
    if prediction == 1:
        return 'negative'
    else:
        return 'positive'

In [11]:
txt = "bad product, i hate it"
preprocessed_text = preprocessing(txt)
vectorized_text = vectorizer(preprocessed_text, tokens)
prediction = get_prediction(vectorized_text)
prediction

'negative'